In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from astropy.table import Table, Column, vstack, hstack, unique, SortedArray,SCEngine
import astropy.units as u

from astropy.io import fits, ascii

import glob
import os                                               


In [3]:
def calcsignoise(subset = False, path = "/Volumes/GoogleDrive/My Drive/andes (1)/tiles/",
                 outfile='/Volumes/GoogleDrive/My Drive/Huge_Table.fits',
                 plot=True, verbose=False):

    all_files = glob.glob(os.path.join(path,"*/*/coadd*fits"))  # takes the each spectra file
    z_files = glob.glob(os.path.join(path,"*/*/zbest*fits"))  # takes each zbest files

    if subset:
        sub_files = all_files[0:10]  # takes the first 10 files to start off
        sub_zfiles = z_files[0:10]
    else:
        sub_files = all_files
        sub_zfiles = z_files

    sub_files.sort()   # sorts each file numerically
    sub_zfiles.sort()

    new_tables = []   # creates an empty table
    for i in range(0,100):  # to go through each file, we need a for loop
        coadd = Table.read(sub_files[i]) # reads the i file table
        hdul = fits.open(sub_files[i])  # opens the fit data that belongs to the i sub_file and gets the information from that file

        zbest = Table.read(sub_zfiles[i])  # reads the i file table
        hdul_z = fits.open(sub_zfiles[i])  # opens the fit data that belongs to the i sub_zfile and gets the information from that file

        r_wave = np.array(hdul['R_WAVELENGTH'].data)  # Takes the chosen row of the hdul file
        r_flux = np.array(hdul['R_FLUX'].data)  # Takes the chosen row of the hdul file
        r_ivar = np.array(hdul['R_IVAR'].data)  # Takes the chosen row of the hdul file

        
        median_array = np.array([])  # this creates an empty array to use later

        for n in range(0,500):  # this is to create the plot. A for loop is needed to go through each array and plot wavelength vs s/n
            rflux = r_flux[n,:]  # takes the i flux value one column at a time
            rivar = r_ivar[n,:]  # takes the i ivar value one column at a time

            r_var = 1/rivar  # inverts the ivar to get var
            r_sigma = np.sqrt(r_var)  # takes the var values and takes the sqaure root of each of the values in the array
            r_sig_noise = rflux/r_sigma # divides flux by sigma to get S/N

            if plot:
                plt.plot(r_wave, r_sig_noise)  # this plots the S/N against the wavelength

            r_SN_med = np.median(r_sig_noise)  # this takes the median S/N to the plot
            median_array = np.append(median_array, r_SN_med)  # this appends the median S/N to the empty array created earlier


        if plot:
            plt.title('Spectrum %s' %i, fontsize = 15)  #  places a title and sets font size
            plt.xlabel('Wavelength', fontsize = 15)   # places a label on the x axis and sets font size
            plt.ylabel('S/N', fontsize = 15) # places a label on the y axis and sets font size

            plt.show()

        z = Table()     # creates an empty table
        z['Z'] = zbest['Z']   # adds this column from zbest table to the empty table we created
        z['ZWARN'] = zbest['ZWARN']   # adds this column from zbest table to the empty table we created
        z['SPECTYPE'] = zbest['SPECTYPE']   # adds this column from zbest table to the empty table we created

        t = Table(coadd)    # takes the table spec and assigns to t
        t['S_N_r'] = median_array  # adds the median S/N of each graph and adds it to t

        table = hstack([t, z])   # combines both z and t to one table

        new_tables.append(table)  # this appends the table we made above to our empty table "new_tables"

    all_tables = vstack(new_tables)  # stacks each of the table together vertically

    # Print?
    if verbose:
        print(all_tables)

    # Write
    all_tables.write(outfile, overwrite=True)

    t_table = Table.read('/Volumes/GoogleDrive/My Drive/Huge_Table.fits')

    print(t_table)


# Command line execution
if __name__ == '__main__':
    calcsignoise(subset= False, path='/Volumes/GoogleDrive/My Drive/andes (1)/tiles/',
                 outfile='/Volumes/GoogleDrive/My Drive/Huge_Table.fits', plot=False)



/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in true_divide


     TARGETID      PETAL_LOC DEVICE_LOC ... ZWARN SPECTYPE
------------------ --------- ---------- ... ----- --------
 35190982328718648         0        459 ...  1570     STAR
 35190982332908466         0        521 ...     0   GALAXY
 35190982332908652         0        522 ...     0   GALAXY
 35190982332908942         0        524 ...  1024   GALAXY
 35190982332909221         0        508 ...     0   GALAXY
 35190982332909422         0        525 ...  1570     STAR
 35190982332909516         0        526 ...  1570     STAR
 35190982332909547         0        509 ...     0   GALAXY
 35190982332909789         0        511 ...  1024   GALAXY
 35190982332909842         0        491 ...     4   GALAXY
               ...       ...        ... ...   ...      ...
611647225851806728         0        116 ...     0   GALAXY
611647225856000836         0        112 ...     4   GALAXY
611647231031771142         0        105 ...     4   GALAXY
611647231031771223         0        107 ...     0   GALA

In [4]:
obs = Table.read('/Volumes/GoogleDrive/My Drive/Huge_Table.fits')


In [5]:
obs

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,PMRA_IVAR,PMDEC_IVAR,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,NUMTARGET,PRIORITY,SUBPRIORITY,OBSCONDITIONS,NUMOBS_MORE,RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,TARGET_RA_IVAR,TARGET_DEC_IVAR,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FRACDEV,FRACDEV_IVAR,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEDEV_R_IVAR,SHAPEDEV_E1_IVAR,SHAPEDEV_E2_IVAR,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,SHAPEEXP_R_IVAR,SHAPEEXP_E1_IVAR,SHAPEEXP_E2_IVAR,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,WISEMASK_W1,WISEMASK_W2,MASKBITS,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PHOTSYS,CMX_TARGET,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,BLOBDIST,FIBERFLUX_IVAR_G,FIBERFLUX_IVAR_R,FIBERFLUX_IVAR_Z,DESI_TARGET,BGS_TARGET,MWS_TARGET,NUM_ITER,FIBER_X,FIBER_Y,MEAN_DELTA_X,MEAN_DELTA_Y,FIBER_RA,FIBER_DEC,NIGHT,EXPID,MJD,TILEID,COADD_NUMEXP,RMS_DELTA_X,RMS_DELTA_Y,FIRST_NIGHT,LAST_NIGHT,NUM_NIGHT,FIRST_EXPID,LAST_EXPID,NUM_EXPID,FIRST_TILEID,LAST_TILEID,NUM_TILEID,FIRST_FIBER,LAST_FIBER,NUM_FIBER,S_N_r,Z,ZWARN,SPECTYPE
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int64,uint8,bytes3,float32,float32,int16,int32,float64,int32,int32,int16,int32,bytes8,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,int64,bytes2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,bytes1,int64,int64,int64,int64,float32,float32,float32,float32,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,int32,int32,float64,int32,int16,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,bytes6
35190982328718648,0,459,459,231,512,104.55545725101197,36.60138233876448,4.900216,-5.4885893,42.29024,47.353317,2015.5,5400.0,545,1,TGT,90.778465,-351.59277,0,59,0.7745636402324717,7,14,8001,527428,1044p365,5432,PSF,1695740200000000.0,1475938800000000.0,0.094205886,58.82356,169.43456,284.13696,229.3759,22.628628,27.123669,0.75663936,0.82873887,0.90025735,1.0721981e-05,3.826281e-06,3.7990158e-06,0.0005179302,0.23572497,0.028075755,0.99999994,0.9999999,0.99999976,6,2,4,1078.5791,236.00189,168.48183,823.558,161.90184,86.61641,135.1858,64.425224,29.44825,257.96872,2.8679764,0.52062815,0.0008355128,1.2016518e-05,0.9841617,0.99024326,0.9979111,0.9992107,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.683174,131.58519,220.66464,45.683174,131.58519,220.66464,0,0,0,940753899756726528,G2,17.070635,860.8483,17.764786,113.19001,16.276514,201.20639,1.2721139,0.0,True,0.16078776,True,0.8088824,94.96686,N,545,59,1,6684,0.0,0.0,0.0,0.0,0,0,0,3,91.095,-349.886,-0.37

In [6]:
idx = obs['SPECTYPE'] == 'GALAXY'  
idx_1 = idx & (obs['Z'] < 0.3) 

In [7]:
obs_idx = obs[idx_1]

In [8]:
sort = obs_idx[np.argsort(obs_idx['S_N_r'])]

In [9]:
sort

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,PMRA_IVAR,PMDEC_IVAR,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,NUMTARGET,PRIORITY,SUBPRIORITY,OBSCONDITIONS,NUMOBS_MORE,RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,TARGET_RA_IVAR,TARGET_DEC_IVAR,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FRACDEV,FRACDEV_IVAR,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEDEV_R_IVAR,SHAPEDEV_E1_IVAR,SHAPEDEV_E2_IVAR,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,SHAPEEXP_R_IVAR,SHAPEEXP_E1_IVAR,SHAPEEXP_E2_IVAR,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,WISEMASK_W1,WISEMASK_W2,MASKBITS,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PHOTSYS,CMX_TARGET,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,BLOBDIST,FIBERFLUX_IVAR_G,FIBERFLUX_IVAR_R,FIBERFLUX_IVAR_Z,DESI_TARGET,BGS_TARGET,MWS_TARGET,NUM_ITER,FIBER_X,FIBER_Y,MEAN_DELTA_X,MEAN_DELTA_Y,FIBER_RA,FIBER_DEC,NIGHT,EXPID,MJD,TILEID,COADD_NUMEXP,RMS_DELTA_X,RMS_DELTA_Y,FIRST_NIGHT,LAST_NIGHT,NUM_NIGHT,FIRST_EXPID,LAST_EXPID,NUM_EXPID,FIRST_TILEID,LAST_TILEID,NUM_TILEID,FIRST_FIBER,LAST_FIBER,NUM_FIBER,S_N_r,Z,ZWARN,SPECTYPE
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int64,uint8,bytes3,float32,float32,int16,int32,float64,int32,int32,int16,int32,bytes8,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,int64,bytes2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,bytes1,int64,int64,int64,int64,float32,float32,float32,float32,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,int32,int32,float64,int32,int16,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,bytes6
611651768337564078,3,59,3059,1535,0,104.34772678687987,38.2123211948232,0.0,0.0,0.0,0.0,0.0,5400.0,4294967296,4,SKY,125.09706,52.65751,0,0,0.9695294788321682,63,0,8001,535464,1042p382,430,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00036636964,0.0082493005,-0.024913125,0.0,0.0,0.0,0,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True,0.0,0.0,,4294967296,0,-1,6687,14.92601,14192.321,2663.3315,496.39407,4294967296,0,0,3,125.023,52.612,-0.0015,-0.0115,0.0,0.0,20200305,53174,58914.14210419,63075,2,0.004743416490252569,0.013209844813622905,20200305,20200305,1,53174,53175,2,63075,63075,1,1535,1535,1,-1.6445136070251465,0.2814996079196238,4,GALAXY
611651753984656039,1,77,1077,725,0,104.47388924279788,37.5185195905362,0.0,0.0,0.0,0.0,0.0,5400.0,4294967296,4,SKY,102.44369,-117.27928,0,0,0.9617

In [10]:
sort.write('/Volumes/GoogleDrive/My Drive/HighS_N.fits', overwrite=True)

In [11]:
sort__table = Table.read('/Volumes/GoogleDrive/My Drive/HighS_N.fits')

In [12]:
sort__table

TARGETID,PETAL_LOC,DEVICE_LOC,LOCATION,FIBER,FIBERSTATUS,TARGET_RA,TARGET_DEC,PMRA,PMDEC,PMRA_IVAR,PMDEC_IVAR,REF_EPOCH,LAMBDA_REF,FA_TARGET,FA_TYPE,OBJTYPE,FIBERASSIGN_X,FIBERASSIGN_Y,NUMTARGET,PRIORITY,SUBPRIORITY,OBSCONDITIONS,NUMOBS_MORE,RELEASE,BRICKID,BRICKNAME,BRICK_OBJID,MORPHTYPE,TARGET_RA_IVAR,TARGET_DEC_IVAR,EBV,FLUX_G,FLUX_R,FLUX_Z,FLUX_IVAR_G,FLUX_IVAR_R,FLUX_IVAR_Z,MW_TRANSMISSION_G,MW_TRANSMISSION_R,MW_TRANSMISSION_Z,FRACFLUX_G,FRACFLUX_R,FRACFLUX_Z,FRACMASKED_G,FRACMASKED_R,FRACMASKED_Z,FRACIN_G,FRACIN_R,FRACIN_Z,NOBS_G,NOBS_R,NOBS_Z,PSFDEPTH_G,PSFDEPTH_R,PSFDEPTH_Z,GALDEPTH_G,GALDEPTH_R,GALDEPTH_Z,FLUX_W1,FLUX_W2,FLUX_W3,FLUX_W4,FLUX_IVAR_W1,FLUX_IVAR_W2,FLUX_IVAR_W3,FLUX_IVAR_W4,MW_TRANSMISSION_W1,MW_TRANSMISSION_W2,MW_TRANSMISSION_W3,MW_TRANSMISSION_W4,ALLMASK_G,ALLMASK_R,ALLMASK_Z,FRACDEV,FRACDEV_IVAR,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEDEV_R_IVAR,SHAPEDEV_E1_IVAR,SHAPEDEV_E2_IVAR,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,SHAPEEXP_R_IVAR,SHAPEEXP_E1_IVAR,SHAPEEXP_E2_IVAR,FIBERFLUX_G,FIBERFLUX_R,FIBERFLUX_Z,FIBERTOTFLUX_G,FIBERTOTFLUX_R,FIBERTOTFLUX_Z,WISEMASK_W1,WISEMASK_W2,MASKBITS,REF_ID,REF_CAT,GAIA_PHOT_G_MEAN_MAG,GAIA_PHOT_G_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_MEAN_MAG,GAIA_PHOT_BP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_RP_MEAN_MAG,GAIA_PHOT_RP_MEAN_FLUX_OVER_ERROR,GAIA_PHOT_BP_RP_EXCESS_FACTOR,GAIA_ASTROMETRIC_EXCESS_NOISE,GAIA_DUPLICATED_SOURCE,GAIA_ASTROMETRIC_SIGMA5D_MAX,GAIA_ASTROMETRIC_PARAMS_SOLVED,PARALLAX,PARALLAX_IVAR,PHOTSYS,CMX_TARGET,PRIORITY_INIT,NUMOBS_INIT,HPXPIXEL,BLOBDIST,FIBERFLUX_IVAR_G,FIBERFLUX_IVAR_R,FIBERFLUX_IVAR_Z,DESI_TARGET,BGS_TARGET,MWS_TARGET,NUM_ITER,FIBER_X,FIBER_Y,MEAN_DELTA_X,MEAN_DELTA_Y,FIBER_RA,FIBER_DEC,NIGHT,EXPID,MJD,TILEID,COADD_NUMEXP,RMS_DELTA_X,RMS_DELTA_Y,FIRST_NIGHT,LAST_NIGHT,NUM_NIGHT,FIRST_EXPID,LAST_EXPID,NUM_EXPID,FIRST_TILEID,LAST_TILEID,NUM_TILEID,FIRST_FIBER,LAST_FIBER,NUM_FIBER,S_N_r,Z,ZWARN,SPECTYPE
int64,int16,int32,int64,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,int64,uint8,bytes3,float32,float32,int16,int32,float64,int32,int32,int16,int32,bytes8,int32,bytes4,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,uint8,uint8,int16,int64,bytes2,float32,float32,float32,float32,float32,float32,float32,float32,bool,float32,bool,float32,float32,bytes1,int64,int64,int64,int64,float32,float32,float32,float32,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,int32,int32,float64,int32,int16,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,int64,bytes6
611651768337564078,3,59,3059,1535,0,104.34772678687987,38.2123211948232,0.0,0.0,0.0,0.0,0.0,5400.0,4294967296,4,SKY,125.09706,52.65751,0,0,0.9695294788321682,63,0,8001,535464,1042p382,430,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00036636964,0.0082493005,-0.024913125,0.0,0.0,0.0,0,0,0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,0.0,True,0.0,0.0,,4294967296,0,-1,6687,14.92601,14192.321,2663.3315,496.39407,4294967296,0,0,3,125.023,52.612,-0.0015,-0.0115,0.0,0.0,20200305,53174,58914.14210419,63075,2,0.004743416490252569,0.013209844813622905,20200305,20200305,1,53174,53175,2,63075,63075,1,1535,1535,1,-1.6445136070251465,0.2814996079196238,4,GALAXY
611651753984656039,1,77,1077,725,0,104.47388924279788,37.5185195905362,0.0,0.0,0.0,0.0,0.0,5400.0,4294967296,4,SKY,102.44369,-117.27928,0,0,0.9617